In [1]:
import os
import warnings
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.models import load_model

warnings.filterwarnings('ignore')
NUM_SAMPLES = 1

In [2]:
def load_data(_input_dir, _output_dir="output", _data_size=-1):
    """
    :param _input_dir: input directory name
                      AWS S3 directory name, where the input files are stored
    :param _output_dir: output directory name
                      AWS S3 directory name, where the output files are saved
    :param _data_size: size of data
                      Data size, that needs to be tested, by default it takes value of
                      -1, which means consider all the data
    :return:
            the processed data, and demand data
    """
    import pandas as pd
    from pandas import DataFrame

    # load all the data
    months = ["apr", "may", "jun", "jul", "aug", "sep"]
    file_format = "uber-processed-data-{}14.csv"
    _data = DataFrame()
    for month in months:
        file_name = _input_dir + "/" + file_format.format(month)
        df_sub = pd.read_csv(file_name)
        _data = _data.append(df_sub)

    # sample the data
    if _data_size > 0:
        _data = _data.sample(n=_data_size)

    # summarizing demand data
    _demand = (_data.groupby(['zip']).count()['Date/Time']).reset_index()
    _demand.columns = ['Zip', 'Number of Trips']

    _demand_w = (_data.groupby(['zip', 'weekday']).count()['Date/Time']).reset_index()
    _demand_w.columns = ['Zip', 'Weekday', 'Number of Trips']

    _demand_h = (_data.groupby(['zip', 'hour']).count()['Date/Time']).reset_index()
    _demand_h.columns = ['Zip', 'Hour', 'Number of Trips']

    _demand_wh = (_data.groupby(['zip', 'weekday', 'hour']).count()['Date/Time']).reset_index()
    _demand_wh.columns = ['Zip', 'Weekday', 'Hour', 'Number of Trips']

    return _data, _demand, _demand_w, _demand_h, _demand_wh

In [3]:
class DemandPredictorBase(object):
    """
        Base class for demand predictor
    """

    def __init__(self, _x, _y, _prefix, train=True, output_dir="."):
        self.x = _x
        self.y = _y
        self.prefix = _prefix
        self.output_dir = output_dir
        self.model = self.build_model()
        if train:
            self.train()

    def build_model(self):
        raise NotImplementedError

    def train(self):
        raise NotImplementedError

    def predict(self, _x_test):
        """
        :param _x_test: test dataset
        :return: prediction for the given test dataset _x_test
        """
        return self.model.predict(_x_test)

    def predict_and_scale(self, _x_test, _y_scalar):
        """
        :param _x_test: test dataset
        :param _y_scalar: Scaler
        :return: prediction for the given test dataset _x_test, scaled to the scalar
        """
        return _y_scalar.inverse_transform(self.predict(_x_test))

    @staticmethod
    def get_mse(_y_test, _y_pred):
        """
        :param _y_test: actual test values
        :param _y_pred: predicted test values
        :return: return the mean square error
        """
        return mean_squared_error(_y_test, _y_pred)
    
    def save_model(self, _data_size, _model_id):
        if self.model is not None:
            import s3fs
            from pickle import dump
            fs = s3fs.S3FileSystem(anon=False)
            bucket = f"{self.output_dir}"
            file_name = f"{self.prefix}_model_{_data_size}_{_model_id}.pickle"
            dump(self.model, fs.open(f"s3://{bucket}/{file_name}", 'wb'))


class DemandPredictorNN(DemandPredictorBase):
    def __init__(self, _x, _y, train=True, output_dir=".", hidden_layer_conf=None, d_key="", dt_size=-1):
        self.input_shape = len(_x[0])
        self.output_shape = len(_y[0])
        self.hidden_layer_conf = hidden_layer_conf
        self.d_key = d_key
        self.dt_size = dt_size
        self.epochs = 100
        self.batch_size = 150
        self.verbose = 0
        self.validation_split = 0.2
        self.learning_rate = 0.01
        self.history = None
        super(DemandPredictorNN, self).__init__(_x, _y, "nn", train, output_dir)

    def build_model(self):
        """
            build the model
        """

        model = Sequential()
        model.add(InputLayer(input_shape=(self.input_shape,)))
        if self.hidden_layer_conf is None:
            model.add(Dense(168, activation='relu'))
            model.add(Dropout(0.1))
            model.add(Dense(24, activation='relu'))
            model.add(Dropout(0.01))
            model.add(Dense(self.output_shape, activation='linear'))
            model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        else:
            for layer in self.hidden_layer_conf:
                model.add(Dense(layer["neurons"], activation=layer["activation"]))
        model.add(Dense(self.output_shape, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def predict(self, _x_test):
        if os.path.exists(self.output_dir + f"/best_model_{self.d_key}_{self.dt_size}.h5"):
            self.model = load_model(self.output_dir + f"/best_model_{self.d_key}_{self.dt_size}.h5")
        return super(DemandPredictorNN, self).predict(_x_test)

    def train(self):
        """
            train the model
        """
        self.history = \
            self.model.fit(
                self.x, self.y,
                epochs=self.epochs, batch_size=self.batch_size,
                verbose=self.verbose, validation_split=self.validation_split,
                use_multiprocessing=True
            )

    def save_model(self, _data_size, _model_id):
        if self.model is not None:
            import io
            import s3fs
            model_data = io.BytesIO()
            self.model.save(model_data)
            model_data.seek(0)
            s3 = s3fs.S3FileSystem(anon=False)  # Uses default credentials
            bucket = f"{self.output_dir}"
            file_name = f"{self.prefix}_model_{_data_size}_{_model_id}.h5"
            with s3.open(f's3://{bucket}/{file_name}', 'wb') as f:
                f.write(model_data.getbuffer())  




In [7]:
def transform_data(_processed_data):
    columns = list(_processed_data.columns)
    columns.remove("Number of Trips")
    sc_x = StandardScaler()
    sc_y = MinMaxScaler()
    x = np.array(
        [
            [entry[col] for col in columns]
            for _, entry in _processed_data.iterrows()
        ]
    )
    y = np.transpose([_processed_data["Number of Trips"].to_list()])
    x = sc_x.fit_transform(x)
    y = sc_y.fit_transform(y)
    return x, y, sc_x, sc_y

def generate_full_data(_processed_data, sc_x):
    day_of_week = [w for w in range(7)]
    hour_of_day = [h for h in range(24)]
    zip_codes = list(_processed_data["Zip"].unique())
    from pandas import DataFrame
    from itertools import product
    df_full_sample = DataFrame(list(product(zip_codes, day_of_week, hour_of_day)),
                                  columns=['Zip', 'Weekday', 'Hour'])
    columns = list(df_full_sample.columns)
    x = np.array(
        [
            [entry[col] for col in columns]
            for _, entry in df_full_sample.iterrows()
        ]
    )
    x = sc_x.fit_transform(x)
    return df_full_sample, x

def predict_using_neural_network(_processed_data, _demand_key, _output_dir, _hidden_layer_config, _data_size):
    """
    :param _processed_data: processed_data
    :param _demand_key: demand data type
    :param _output_dir: output directory
    :param _hidden_layer_config: hidden layer configuration
    :param _data_size: data size
    :return: run NUM_SAMPLES time neural network and compute the average MSE and MSE ratio
    """
    _x, _y, _sc_x, _sc_y = transform_data(_processed_data)
    nn = DemandPredictorNN(
        _x=_x, _y=_y,
        output_dir=_output_dir, hidden_layer_conf=_hidden_layer_config,
        d_key=_demand_key, dt_size=_data_size
    )
    
    _initial_df, _x_full_sample = generate_full_data(_processed_data, _sc_x)
    _y_full_sample = nn.predict(_x_full_sample)
    _y_full_sample = _sc_y.inverse_transform(_y_full_sample)
    _y_full_sample_int = [int(_y / 26.1) if _y >= 0 else 0 for _y in _y_full_sample]
    _y_full_sample_float = [float(_y / 26.1) if _y >= 0 else 0 for _y in _y_full_sample]
    _initial_df["Expected Trips"] = _y_full_sample_int
    _initial_df["Expected Trips (Est)"] = _y_full_sample_float
    _initial_df.to_csv(f"{_output_dir}/full_processed_sample_{_demand_key}_{_data_size}.csv", index=False)


In [8]:
def main_function(_data_size, _input_dir, _output_dir):
    """
    :param _data_size: size of the input data
    :param _input_dir: input directory
    :param _output_dir: output directory
    """
    start = datetime.now()
    data, _, _, _, demand_wh = load_data(
        _input_dir=_input_dir, _data_size=_data_size, _output_dir=_output_dir
    )
    end = datetime.now()
    print(f"Data Loading Time : {(end - start).total_seconds()}")
    nn_hidden_layer_config = {
        "weekday_n_hour": [
            {
                "activation": "relu", "neurons": 24
            }
        ]
    }
    demand_data = {
        "weekday_n_hour": demand_wh
    }
    for demand_key in demand_data:
        demand_datum = demand_data[demand_key]
        predict_using_neural_network(
            demand_datum, demand_key, _output_dir=_output_dir,
            _hidden_layer_config=nn_hidden_layer_config[demand_key], _data_size=_data_size
        )

In [9]:
data_size = 100
# provide the custom bucket name where the data is stored
s3_main_bucket_name = "cloud-project-x"
# notebook instance location
notebook_instance_id = "xlarge-model-test"
input_dir = f"s3://{s3_main_bucket_name}"
output_dir = f"s3://{s3_main_bucket_name}/output_{data_size}_{notebook_instance_id}"
main_function(data_size, input_dir, output_dir)

Data Loading Time : 10.510104
